报社等相关的机构，往往会遇到一个问题，就是别人家的机构使用自己的文章但是并没有标明来源。将解决新华社的文章被抄袭引用的问题。

In [1]:
import re
import time
import os
import joblib
import numpy as np
import pandas as pd
from scipy.sparse import save_npz, load_npz
import jieba
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_curve, precision_score, recall_score, f1_score, accuracy_score

# 预处理原始文本

In [4]:
filename = 'datasets/news_from_where.csv'
rawdata = pd.read_csv(filename, encoding='utf-8')

print(len(rawdata))
print(len(rawdata[rawdata['source'] == '新华社']))
rawdata.head()

89611
78661


,id,author,source,content,feature,title,url
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm
1,89616,NaN,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm
2,89615,NaN,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""18""...",一加手机5细节曝光：3300mAh、充半小时用1天,http://www.cnbeta.com/articles/tech/623601.htm
3,89614,NaN,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图）,http://world.huanqiu.com/hot/2017-06/10866126....
4,89613,胡淑丽_MN7479,深圳大件事,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,"{""type"":""新闻"",""site"":""网易热门"",""commentNum"":""978"",...",44岁女子约网友被拒暴雨中裸奔 交警为其披衣相随,http://news.163.com/17/0618/00/CN617P3Q0001875...


- 不平衡数据，新华社的数据占据中数据的 85% 以上

In [5]:
# 语料数据和标签
def get_data(rawdata):
    mask = rawdata['content'].notnull()
    data = rawdata.loc[mask, ['source', 'content']]
    return data


data = get_data(rawdata)
print(data.shape)

data.head()

(87054, 2)


,source,content
0,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...
1,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...
2,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...
3,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n
4,深圳大件事,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...


In [6]:
# 预料数据的统计信息
data.content.str.len().describe()

count    87054.000000
mean       444.887380
std        670.024847
min          3.000000
25%        132.000000
50%        176.000000
75%        499.000000
max      22422.000000
Name: content, dtype: float64

In [7]:
data.content[0]

'此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/体验版内测，稳定版暂不受影响），以确保工程师可以集中全部精力进行系统优化工作。有人猜测这也是将精力主要用到MIUI 9的研发之中。\r\nMIUI 8去年5月发布，距今已有一年有余，也是时候更新换代了。\r\n当然，关于MIUI 9的确切信息，我们还是等待官方消息。\r\n'

In [8]:
data.content[89609]

'\\n\\n2017年5月25日，在美国马萨诸塞州剑桥市，哈佛大学毕业生在毕业典礼上欢呼。（新华/欧新）;;;;新华社北京6月7日电\\n（\\n记者\\n夏文辉）\\n据美国哈佛大学的知名校园媒体《哈佛深红色》消息，哈佛大学取消了至少10名新生的入学资格，因为他们在社交媒体脸书上发表了涉及性、种族歧视、儿童虐待及极端主义等不当内容。美联社等媒体5日转引了这一报道。\\n据报道，这些学生去年12月在脸书上建立了一个群，用于发布和共享信息，一些内容不堪入目。比如，一篇文章转载了一个死于叙利亚轰炸的儿童的血腥照片，图片不仅没有打码，不少学生还对此恶意戏谑。\\n据《哈佛深红色》报道，今年4月，哈佛大学招生部门向部分学生发函，要求他们解释在社交媒体上发布的极端和不当内容，并表示要审核他们的入学情况。校方同时通知他们不必参加4月新生的到校访问活动。大约一周后，至少10名学生接到通知，他们的入学资格被哈佛大学取消。\\n《哈佛深红色》没有给出未能入学学生的名字。美联社也未能联系到具体学生。哈佛大学发言人理查德·戴恩回复路透社采访询问说，校方不会公开讨论申请入学者情况。\\n按照哈佛大学的规定，以下几种情况有可能被学校取消入学资格：高中未能毕业、入学申请造假，以及申请人存在有违正直、诚实等品行的情况。\\n'

In [11]:
# 无效的数据
null_data = rawdata.loc[rawdata.content.isnull()]
print(len(null_data))
null_data.head(2)

2557


,id,author,source,content,feature,title,url
100,89517,NaN,中国证券报?中证网,NaN,"{""type"":""公司"",""site"":""中证网"",""commentNum"":""0"",""jo...",天和防务股东未来6个月内计划减持不超过480万股公司股份,http://www.cs.com.cn/ssgs/gsxw/201706/t2017062...
103,89514,NaN,中国证券报?中证网,NaN,"{""type"":""公司"",""site"":""中证网"",""commentNum"":""0"",""jo...",晶盛机电调整限制性股票回购价格,http://www.cs.com.cn/ssgs/gsxw/201706/t2017062...


# 数据处理及向量化

In [12]:
def preprocessor(doc):
    """删除文本中所有非中文词及常用标点"""
    regx = re.compile('[^\u4e00-\u9fa5‘’“”，、；。？！《》（）.0123456789%// A-Za-z]')
    doc = regx.sub('', doc)
    return doc


def data2matrix(data):
    """文本向量化"""
    y = data['source'].apply(lambda s: 1 if s == "新华社" else 0)
    vectorizer = TfidfVectorizer(tokenizer=jieba.cut,
                                 preprocessor=preprocessor,
                                 max_df=0.8,
                                 max_features=10000)
    X = vectorizer.fit_transform(data['content'])
    return X, y, vectorizer

In [13]:
%%time
X, y, vectorier = data2matrix(data)

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.412 seconds.
Prefix dict has been built successfully.


CPU times: user 1min 27s, sys: 123 ms, total: 1min 27s
Wall time: 1min 27s


In [15]:
# 不平衡数据，不论什么新闻总是判断为新华社，精度也达到 90%
from sklearn.base import BaseEstimator


class AlwaysXinHuaShe(BaseEstimator):
    def fit(self, x, y=None):
        pass

    def predict(self, x):
        return (np.ones((len(x), 1), dtype=bool))


print("Accuracy for AlwaysXinHuaShe {}".format(len(y[y == 1.]) / len(y)))

Accuracy for AlwaysXinHuaShe 0.9035885772049532


In [24]:
vectorier.vocabulary_

{'此外': 6070,
 '自': 8026,
 '本周': 5752,
 '起': 8818,
 '除': 9566,
 '小米': 3781,
 '手机': 4713,
 '外': 3136,
 '其余': 1577,
 '已': 3978,
 '暂停': 5596,
 '更新': 5618,
 '发布': 2383,
 '含': 2634,
 '开发': 4228,
 '版': 6711,
 '影响': 4362,
 '以': 989,
 '确保': 7241,
 '工程师': 3954,
 '可以': 2485,
 '集中': 9613,
 '全部': 1481,
 '精力': 7595,
 '进行': 9100,
 '工作': 3936,
 '有人': 5668,
 '猜测': 6784,
 '这': 9050,
 '也': 712,
 '是': 5553,
 '将': 3751,
 '主要': 643,
 '研发': 7219,
 '之中': 668,
 '去年': 2302,
 '一年': 61,
 '时候': 5488,
 '了': 741,
 '当然': 4341,
 '关于': 1556,
 '信息': 1289,
 '我们': 4642,
 '还是': 9047,
 '等待': 7525,
 '作为': 1178,
 '唯一': 2730,
 '通过': 9210,
 '平台': 4089,
 '认证': 8501,
 '强调': 4315,
 '不会': 289,
 '因为': 2815,
 '只': 2475,
 '考虑': 7921,
 '性能': 4481,
 '而': 7926,
 '去': 2300,
 '小': 3758,
 '核心': 5940,
 '相反': 7113,
 '他们': 975,
 '正': 6050,
 '联手': 7960,
 '微软': 4414,
 '找到': 4789,
 '一种': 101,
 '适合': 9155,
 '、': 4,
 '兼顾': 1598,
 '和': 2687,
 '完美': 3578,
 '方案': 5393,
 '报道': 4865,
 '称': 7361,
 '已经': 3981,
 '拿到': 4943,
 '一些': 16,
 '新': 5339,
 '以便': 99

# 构建机器学习模型

- 拆分数据集为 train、test

In [16]:
x_train, x_test, y_train, y_test = train_test_split(X, y) 

x_train.shape,x_test.shape

((65290, 10000), (21764, 10000))

In [17]:
len(y_train[y_train==1])/len(y_train)

0.9036759074896615

## **`KNN`分类器**

In [18]:
def build_model(clf, x, y, params):
    model = GridSearchCV(clf, cv=3, param_grid=params)
    model.fit(x, y)
    return model

clf = KNeighborsClassifier()
params = {'n_neighbors':range(2,5), 'weights':('uniform', 'distance')}

In [23]:
%%time
model = build_model(clf, x_train, y_train, params)

CPU times: user 17min 26s, sys: 57.4 s, total: 18min 24s
Wall time: 10min 43s


In [24]:
knn = model.best_estimator_
knn

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                     weights='distance')

In [25]:
def model_performance(model, x_test, y_test):
    y_pred = model.predict(x_test)
    print('accuracy is: ', accuracy_score(y_test, y_pred))
    print('precision is: ', precision_score(y_test, y_pred))
    print('recall is: ', recall_score(y_test, y_pred))
    print('f1_score is: ', f1_score(y_test, y_pred))

model_performance(knn, x_test, y_test)

accuracy is:  0.8924370520124977
precision is:  0.9883268482490273
recall is:  0.8914547304170906
f1_score is:  0.9373946995426953


- 模型分类的准确度为 0.941，即 94.1% 的文章分类正确
- 查准率为0.946， 即模型判断为新华社的文章中，有 5.4% 是误分类
- 查全率为0.991， 即所有新华社的文章，模型将其中的 0.9% 分类错误

## **`SVM`分类**

In [38]:
%%time
svm_clf = SVC()
svm_clf.fit(x_train, y_train)

svm_clf

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [39]:
model_performance(svm_clf, x_test, y_test)

accuracy is:  0.9043374379709612
precision is:  0.9043374379709612
recall is:  1.0
f1_score is:  0.9497659605269507


- `SVM`分类器精度较低，与全猜为新华社的效果相当；该分类器基本无效
- 模型可以将新华社的文章全部判断正确
- 原因：不均衡的训练数据？新华社远多于其它来源

## **朴素贝叶斯分类**

In [ ]:
%%time
bayes_clf = GaussianNB()
bayes_clf.fit(x_train, y_train)

In [ ]:
bayes_clf

In [ ]:
model_performance(bayes_clf, x_test, y_test)

## **逻辑回归**

In [47]:
%%time
logit_clf = LogisticRegression()
logit_clf.fit(x_train, y_train)

/home/student/anaconda3/envs/noam/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


CPU times: user 3.13 s, sys: 92 ms, total: 3.22 s
Wall time: 1.65 s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [48]:
model_performance(logit_clf, x_test, y_test)

accuracy is:  0.969031428046315
precision is:  0.9709146764443564
recall is:  0.9955797175083833
f1_score is:  0.9830925145494681


- 逻辑回归效果比较好，速度快

## **随机森林**

In [25]:
%%time
forest = RandomForestClassifier()
forest.fit(x_train, y_train)

/home/student/anaconda3/envs/noam/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


CPU times: user 7.89 s, sys: 88 ms, total: 7.98 s
Wall time: 7.64 s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [28]:
model_performance(forest, x_test, y_test)

accuracy is:  0.9676989523984562
precision is:  0.9728074107276259
recall is:  0.9920264093448451
f1_score is:  0.9823229148331616


- 速度较快
- 效果还行

# 抄袭的文章判别
- 测试文章中，模型判别为新华社，且概率越高，但同时该文章不属于新华社的，抄袭可能性越大

In [60]:
indexes = y_test.index

In [61]:
y_pred = forest.predict(x_test)
y_pred_prob = forest.predict_proba(x_test)

In [62]:
y_pred_with_prob = pd.DataFrame({'label':y_pred,'prob':y_pred_prob[:,1]}, index=indexes)

In [63]:
y_pred_with_prob

,label,prob
59848,1,1.000000
6070,0,0.500000
4176,0,0.500000
58593,1,1.000000
48695,1,1.000000
73288,1,0.900000
72765,1,0.900000
3077,0,0.500000
48236,1,1.000000
5019,0,0.400000


In [65]:
mask1 = y_pred_with_prob[(y_pred_with_prob.label==1.0)&(y_pred_with_prob.prob<0.60)].index
mask1

Int64Index([ 2473,  4993,  5394,  5197,  4539,  6001,  7559,  2974,  4565,
            86202, 86565,  7037,  5174, 45999,  4106,  5949,  6612,   453,
              670,  5309],
           dtype='int64')

In [66]:
mask2 = y_test[y_test==0].index
mask2

Int64Index([ 6070,  4176,  3077,  5019,  6968, 10765,  1023,  4640,   407,
             6446,
            ...
             7349,  3126,  4995,     4,  1881,  4184, 10799,  7402,  7414,
              461],
           dtype='int64', length=2074)

In [68]:
plagiaristic = data.loc[mask1.intersection(mask2)]
len(plagiaristic)

16

In [69]:
plagiaristic

,source,content
4993,广州日报第ZSA15版,中山分类信息\r\n
5394,广州日报第A20版,对联\r\n 上联：爱妻，爱子，爱家庭，不爱身体等于零；\r\n 下联：有钱，有权，有成...
5197,广州日报第FS1版,霍震霆： 我的孩子 特别爱国\r\n 8\r\n
4539,广州日报第A24版,广州日报大收订\r\n
6001,海南日报第010版,海南义隆实业有限公司：\r\n 纳税人识别号：460100730064372\r\n ...
7559,广州日报第01版,无人驾驶来 道路将咋变\r\n
2974,证券时报?e公司,6月15日，央行上海总部公布统计数据显示，5月上海地区个人住房贷款新增154.86亿元，...
4565,央视网,央视网消息：防治土壤污染，直接关系到农产品质量安全、人民群众身体健康和经济社会的可持续发展。...
7037,Wind资,Wind资讯，周四（6月22日），金融股带头，地产股跟风，沪深两市股指涨幅不断扩大，上证...
5174,广州日报第FS1版,新会出土 唐胡人陶俑\r\n 10\r\n


- 部分文章字数太少，无从确定是否抄袭

In [71]:
plagiaristic['content'].str.len()

4993      8
5394    127
5197     21
4539      9
6001    567
7559     13
2974    151
4565    114
7037     97
5174     18
4106      8
5949     93
6612     21
453     164
670     349
5309     18
Name: content, dtype: int64

In [72]:
text1 = plagiaristic.loc[6001]['content']
text1

'\u3000\u3000海南义隆实业有限公司：\r\n\u3000\u3000纳税人识别号：460100730064372\r\n\u3000\u3000事由：你公司在规定期限内未履行已生效的《海南省地方税务局第一稽查局税务处理决定书》（琼地税一稽处〔2014〕22号）和《海南省地方税务局第一稽查局税务行政处罚决定书》（琼地税一稽罚〔2015〕3号）应缴纳的税费，经催缴仍未缴纳，根据《中华人民共和国税收征收管理法》第四十条规定，采取强制执行措施，扣缴税款20,588.00元，拍卖财产抵缴罚款68,150.00元。尚欠税款652,371.49元、滞纳金304,938.95元、罚款183,767.24元，合计1,141,077.68元（因超期限未缴纳税款产生的滞纳金以金三系统计算为准）。\r\n\u3000\u3000因采取其他方式无法送达，根据《中华人民共和国税收征收管理法实施细则》第一百零六条规定，现依法向你公司公告送达《催告通知书》（琼地税一稽催通〔2017〕2号）。自本公告发出之日起满30日，即视为送达。\r\n\u3000\u3000如不服该《催告通知书》，必须根据本催告的期限缴纳税款，然后可自上述款项缴清之日起六十日内依法向海南省地方税务局第一稽查局申请行政复议。逾期本《催告通知书》即发生法律效力。\r\n\u3000\u3000附件：《催告通知书》（琼地税一稽催通〔2017〕2号）\r\n\u3000\u3000海南省地方税务局第一稽查局???2017年6月23日\r\n'

In [73]:
text2 = plagiaristic.loc[670]['content']
text2

'香火钱”是香客捐给寺庙，用于日常供奉香烛等，然而，一些不法分子表面佯装虔诚的信徒，进庙烧香拜佛，背地里却盯上了放在案台上的香火钱。\r\n近日，上海市青浦公安抓获违法人员敬某，当场缴获其盗窃所得。\r\n6月23日，青浦区徐泾镇蟠龙庵举行庙会，赶会香客近万人。此时，一名中年女子混迹香客之中不停地来回转悠，目光还时不时盯向落在供奉菩萨案台上香客们留下的硬币。观察一阵后，该女子佯装拜佛祷告，夹杂在人群中接近案台，乘人不备迅速伸手抓取案台上的硬币放进自己的口袋。如此陆续没过几分钟，案台上的“香火钱”就被该女子悉数收入囊中。让她没想到的是，自己的盗窃行为已被庵内工作人员发现并报警。就在该女子发现情况不妙准备离开庵堂溜之大吉时，被及时赶到的民警抓获。\r\n目前，敬某因盗窃行为已被青浦公安分局依法行政拘留。\r\n'

In [77]:
text3 = plagiaristic.loc[453]['content']
text3

'但 SMBv1 协议实现的漏洞导致了勒索软件 WannaCry 在存在漏洞的系统中广泛传播，然而受影响的系统主要是 Windows 7 而不是微软计划更新的 Windows 10。\r\n微软证实，用户在全新安装系统时候将不会包含 SMBv1，但从现有系统升级时 SMBv1 仍然会留在系统中。微软的这一决定不会影响现有的系统。\r\n'